# Covid-19 JHU Dataset

In [14]:
# Import libraries

import os
import json
import re

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

#plotly libraries
import plotly.express as px
import plotly.graph_objects as go
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
#import cufflinks as cf
#import plotly.plotly as py

import seaborn as sns

In [16]:
# import 4 main data files
countries = pd.read_csv(r'C:\Users\vijay\OneDrive\Documents\Projects\Covid\covid-19\data\countries-aggregated.csv')
countries_pv = pd.read_csv(r'C:\Users\vijay\OneDrive\Documents\Projects\Covid\covid-19\data\key-countries-pivoted.csv')
time_series = pd.read_csv(r'C:\Users\vijay\OneDrive\Documents\Projects\Covid\covid-19\data\time-series-19-covid-combined.csv')
ww_agg = pd.read_csv(r'C:\Users\vijay\OneDrive\Documents\Projects\Covid\covid-19\data\worldwide-aggregated.csv')

Convert to date time

In [23]:
# convert Date object to Datetime
countries['Date'] = pd.to_datetime(countries['Date'])
countries_pv['Date'] = pd.to_datetime(countries_pv['Date'])
time_series['Date'] = pd.to_datetime(time_series['Date'])
ww_agg['Date'] = pd.to_datetime(ww_agg['Date'])

In [26]:
countries

,Date,Country,Confirmed,Recovered,Deaths
0,2020-01-22,Afghanistan,0,0,0
1,2020-01-22,Albania,0,0,0
2,2020-01-22,Algeria,0,0,0
3,2020-01-22,Andorra,0,0,0
4,2020-01-22,Angola,0,0,0
...,...,...,...,...,...
13208,2020-04-03,Venezuela,153,52,7
13209,2020-04-03,Vietnam,237,85,0
13210,2020-04-03,West Bank and Gaza,194,21,1
13211,2020-04-03,Zambia,39,2,1


In [27]:
countries_pv

,Date,China,US,United_Kingdom,Italy,France,Germany,Spain,Iran
0,2020-01-22,548,1,0,0,0,0,0,0
1,2020-01-23,643,1,0,0,0,0,0,0
2,2020-01-24,920,2,0,0,2,0,0,0
3,2020-01-25,1406,2,0,0,3,0,0,0
4,2020-01-26,2075,5,0,0,3,0,0,0
...,...,...,...,...,...,...,...,...,...
68,2020-03-30,82198,161807,22453,101739,45170,66885,87956,41495
69,2020-03-31,82279,188172,25481,105792,52827,71808,95923,44605
70,2020-04-01,82361,213372,29865,110574,57749,77872,104118,47593
71,2020-04-02,82432,243453,34173,115242,59929,84794,112065,50468


In [64]:
time_series2=time_series.fillna(0)
time_series2['Description'] = time_series2['Country/Region']#+ ', ' \
time_series2.head()

,Date,Country/Region,Province/State,Lat,Long,Confirmed,Recovered,Deaths,Description
0,2020-01-22,Afghanistan,0,33.0,65.0,0.0,0.0,0.0,Afghanistan
1,2020-01-23,Afghanistan,0,33.0,65.0,0.0,0.0,0.0,Afghanistan
2,2020-01-24,Afghanistan,0,33.0,65.0,0.0,0.0,0.0,Afghanistan
3,2020-01-25,Afghanistan,0,33.0,65.0,0.0,0.0,0.0,Afghanistan
4,2020-01-26,Afghanistan,0,33.0,65.0,0.0,0.0,0.0,Afghanistan


In [65]:
df = px.data.gapminder()
fig = px.scatter_geo(time_series2,
                     #lat="Lat"
                     #lon="Long"
                     locations="Country/Region", 
                     #color="Country/Region",
                     #size="pop",
                     #mode="markers",
                     hover_name="Description", 
                     size="Confirmed",
                     animation_frame=time_series2["Date"].astype(str),
                     projection="natural earth",
                     template='plotly_dark')
#fig.update_geos(fitbounds="locations")
fig.show()

In [ ]:
lat=df['Latitude'],
                     lon=df['Longitude'],